In [1]:
import os
import tensorflow as tf
import numpy as np
import tvm
import tvm.relay as relay
import tvm.contrib.graph_runtime as runtime
from tvm.relay.expr_functor import ExprMutator
from riptide.get_models import get_model
from riptide.binary.binary_layers import Config, DQuantize, XQuantize

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [8]:
config = Config(actQ=DQuantize, weightQ=XQuantize, bits=2, use_act=False, use_bn=False, use_maxpool=True)

In [9]:
with config:
    model = get_model('vggnet')
#model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Conv2D(filters=96, strides=2, padding='same', kernel_size=7, use_bias=False, data_format='channels_last'))
#model.add(tf.keras.layers.BatchNormalization(center=False, scale=False))

In [10]:
test_input = tf.keras.Input(shape=[32, 32, 3], batch_size=1, dtype='float32')
output = model(test_input)

In [11]:
func, params = relay.frontend.from_keras(model, shape={'input_1': [1, 32, 32, 3]})

In [12]:
with relay.build_config(opt_level=0):
    #graph, lib, params = relay.build(func, 'llvm', params=params)
    graph, lib, params = relay.build_module.build(func, target='llvm', params=params)
    #out = intrp.evaluate(func)(np.random.uniform(size=(1, 3, 28, 28)))

W0610 04:21:24.722153 140568053081920 dispatcher.py:381] Cannot find config for target=llvm, workload=('bitserial_conv2d_nchw', (1, 512, 2, 2, 'int16'), (512, 512, 3, 3, 'int16'), (1, 1), (1, 1), 2, 1, 'uint32', 'int16', 1). A fallback configuration is used, which may bring great performance regression.
W0610 04:21:24.783376 140568053081920 dispatcher.py:381] Cannot find config for target=llvm, workload=('bitserial_conv2d_nchw', (1, 512, 4, 4, 'int16'), (512, 512, 3, 3, 'int16'), (1, 1), (1, 1), 2, 1, 'uint32', 'int16', 1). A fallback configuration is used, which may bring great performance regression.
W0610 04:21:24.840718 140568053081920 dispatcher.py:381] Cannot find config for target=llvm, workload=('bitserial_conv2d_nchw', (1, 256, 4, 4, 'int16'), (512, 256, 3, 3, 'int16'), (1, 1), (1, 1), 2, 1, 'uint32', 'int16', 1). A fallback configuration is used, which may bring great performance regression.
W0610 04:21:24.898673 140568053081920 dispatcher.py:381] Cannot find config for targe

2
2
2
2


W0610 04:21:24.957203 140568053081920 dispatcher.py:381] Cannot find config for target=llvm, workload=('bitserial_conv2d_nchw', (1, 96, 8, 8, 'int16'), (256, 96, 3, 3, 'int16'), (1, 1), (1, 1), 2, 1, 'uint32', 'int16', 1). A fallback configuration is used, which may bring great performance regression.


2


In [8]:
module = runtime.create(graph, lib, tvm.context('llvm', 0))
module.set_input('input_1', np.random.uniform(size=(1, 3, 32, 32)))
module.set_input(**params)

In [9]:
module.run()
print(module.get_output(0).shape)

(1, 1000)
